# 🧠 Brain Tumor Detection using CNN + Gradio

This project builds a deep learning model to **detect brain tumors** in MRI scans using a **Convolutional Neural Network (CNN)**. A user-friendly **Gradio interface** allows users to upload an image and get a prediction: **Tumor** or **No Tumor**.

## 📁 Dataset Used

**[Brain MRI Images for Brain Tumor Detection](https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection)** – from Kaggle

- Contains MRI scans labeled as:
  - ✅ `yes` (tumor present)
  - ❌ `no` (no tumor)


## 🔧 What the Project Does

1. **Fetches dataset directly from Kaggle** using Kaggle API  
2. **Preprocesses MRI images**:
 - Resize to 128x128
 - Normalize pixel values
 - Convert labels to categorical
3. **Builds a CNN model** using TensorFlow/Keras
4. **Trains the model** on the MRI images (80% train, 20% test)
5. **Creates a Gradio web interface** where users can upload MRI scans for prediction


## 🧠 Model Architecture

```text
Input: 128x128 RGB MRI image

Conv2D (32 filters) → MaxPooling  
Conv2D (64 filters) → MaxPooling  
Flatten → Dropout  
Dense (64) → Dense (2 - softmax)


In [ ]:
# 📦 1. Install dependencies
!pip install -q kagglehub gradio tensorflow opencv-python-headless matplotlib

# 📥 2. Download dataset using kagglehub
import kagglehub
import os

dataset_path = kagglehub.dataset_download('navoneel/brain-mri-images-for-brain-tumor-detection')
print('✅ Data source import complete.')


✅ Data source import complete.


In [ ]:
# 🧹 3. Load and preprocess the data
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

categories = ["yes", "no"]
data = []
labels = []

for category in categories:
    path = os.path.join(dataset_path, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            img_array = cv2.resize(img_array, (128, 128))
            data.append(img_array)
            labels.append(class_num)
        except:
            pass

data = np.array(data) / 255.0
labels = to_categorical(labels, 2)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print("✅ Data loaded and preprocessed.")


✅ Data loaded and preprocessed.


In [ ]:
# 🏗️ 4. Build and train the CNN model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))
print("✅ Model training complete.")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 783ms/step - accuracy: 0.6240 - loss: 1.2669 - val_accuracy: 0.8039 - val_loss: 0.5750
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 895ms/step - accuracy: 0.7890 - loss: 0.5491 - val_accuracy: 0.7255 - val_loss: 0.5525
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 699ms/step - accuracy: 0.7761 - loss: 0.5207 - val_accuracy: 0.7647 - val_loss: 0.5413
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 723ms/step - accuracy: 0.8039 - loss: 0.4382 - val_accuracy: 0.8235 - val_loss: 0.4522
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 774ms/step - accuracy: 0.8246 - loss: 0.4543 - val_accuracy: 0.8039 - val_loss: 0.4793
✅ Model training complete.


In [ ]:
# 🧪 5. Create Gradio interface for prediction
import gradio as gr

def predict_tumor(image):
    image = cv2.resize(image, (128, 128)) / 255.0
    image = np.expand_dims(image, axis=0)
    prediction = model.predict(image)[0]
    return {
        "Tumor": float(prediction[0]),
        "No Tumor": float(prediction[1])
    }

gr.Interface(
    fn=predict_tumor,
    inputs=gr.Image(type="numpy", label="Upload Brain MRI"),
    outputs=gr.Label(num_top_classes=2),
    title="🧠 Brain Tumor Detector",
    description="Upload a brain MRI image to detect whether a brain tumor is present.",
    theme="default"
).launch(share=True, debug=True)




Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5c529356fb09e8cf19.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
